<a href="https://colab.research.google.com/github/nwenzel/Everpix-Intelligence/blob/master/sentiment_battle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install langchain langchain_community langchain_core langchain_openai langchain-google-genai

In [ ]:
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm

In [ ]:
# curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
%xterm

In [ ]:
from google.colab import userdata
openai = userdata.get("OPENAI_API_KEY")
google = userdata.get("GOOGLE_API_KEY")
tracing = userdata.get("LANGCHAIN_API_KEY")

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = openai
os.environ["GOOGLE_API_KEY"] = google
os.environ["LANGCHAIN_API_KEY"] = tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "office_hours_4"

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.llms.ollama import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI

got_mail_example = "This film shouldn't work at all. It doesn't have much of a story and the whole dial up internet thing is incredibly dated. However Hanks and Ryan sell it beautifully."

model_one = ChatGoogleGenerativeAI(model="gemini-1.5-flash", max_retries=1)
model_two = Ollama(model="phi3:mini")
model_three = ChatOpenAI(model="gpt-4o")

prompt = ChatPromptTemplate.from_template(
    """Analyze the overall sentiment of the following movie review:

    REVIEW: {review}

    Only return a sentiment of 'positive' or 'negative'. Do not include any other text. You must respond, and respond in a single word.
    """
)

output_parser = StrOutputParser()

runnable_one = prompt | model_one | output_parser
runnable_two = prompt | model_two | output_parser
runnable_three = prompt | model_three | output_parser

parallel = RunnableParallel(
    { "first_review": runnable_one,
    "second_review": runnable_two,
    "review": RunnablePassthrough(),
    }
)

def analyze(results):
    print("analyzing results...")
    if results["first_review"].lower().strip() == results["second_review"].lower().strip:
        return results["first_review"].lower().strip()
    else:
        print("running tie-breaker")
        return runnable_three.invoke({ "review": results["review"] }).lower().strip()

analyze(parallel.invoke({ "review": got_mail_example }))